In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = [12,8]
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
import sys
sys.path.append('../..')

# Mixture models

The  quadratic discriminant analysis made rather strong assumptions that that distributions of features in each class was multivariate normal. Obviously often this assumptions will not hold.  Consider the following synthetic dataset:

In [ ]:
half_circles = np.loadtxt('half_circles.txt')
hc_labels = half_circles[:,2].astype('int32')
hc_data = half_circles[:,:2]

In [ ]:
colors = np.asarray(['blue', 'red'])
plt.scatter(half_circles[:,0], half_circles[:,1], s=30, alpha =0.5, 
            c=colors[half_circles[:,2].astype('int32')]);

Clearly the distributions are not normal the clusters are intertwined and althought they look well separated  we can expect the quadratic discriminant analysis to perform poorly on this data set. Let's  check it out

In [ ]:
from sklearn.model_selection import train_test_split
hc_train,hc_test, hc_lbl_train, hc_lbl_test = train_test_split(hc_data, hc_labels, test_size=0.25)

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
hc_qda = QuadraticDiscriminantAnalysis(store_covariance=True)

In [ ]:
hc_qda.fit(hc_train, hc_lbl_train)

In [ ]:
qda_proba = hc_qda.predict_proba(hc_test)[:,1]

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score

In [ ]:
confusion_matrix(hc_lbl_test, qda_proba>0.5, normalize='true')

In [ ]:
from mchlearn.plotting import roc_plot, add_roc_curve
fig, ax = roc_plot()
add_roc_curve(hc_lbl_test, qda_proba, 'qda', ax);
ax.legend(title='AUC');

So while not totally useless the classifiers performance is not  exactly stellar. It's easy to understand why, when we look at confidence ellipses of fitted distributions and  resulting decision boundaries

In [ ]:
from mchlearn.plotting import confidence_ellipse, decision_plot

In [ ]:
blue_red = np.asarray([[0., 0.,1.],[1.,0.,0.]]) 
decision_plot(half_circles[:,:2], half_circles[:,2].astype('int32'), blue_red, np.linspace(-2.5,2.5,200), np.linspace(-1.5,1.5,200),hc_qda.predict_proba, np.argmax  )
confidence_ellipse(hc_qda.means_[0], hc_qda.covariance_[0], ax = plt.gca(), edgecolor='blue')
confidence_ellipse(hc_qda.means_[1], hc_qda.covariance_[1], ax = plt.gca(), edgecolor='red');

The QDA classifier, constrained to quadratic curve, cannot reproduce the complex boundary between clusters.

## Mixture of Gaussians

A simple generalisation of normal distribution is a _mixture of gaussians_ distribution which is a special case of _mixture model_. As the name implies those distribution consists of a _mixture_ of normal distributions. The resulting probability density function (pdf) is a  weighted sum of the pdfs of each mixture component. I will ilustrate this on a simple mixture of two  univariate (one dimensional) gaussian distribution.

### One dimension

The pdf for a mixture of two  normal distributions is given by the formula 

$$\newcommand{\nc}{\mathcal{N}}$$
$$p(x|p, \mu_0,\sigma_0, \mu_1, \sigma_1)=\pi \cdot p_\nc(x|\mu_1,\sigma_1) +  (1-\pi) \cdot p_\nc(x|\mu_0,\sigma_0) $$

where $p_\nc(x|\mu,\sigma)$ denotes the pdf of normal distribution with mean $\mu$ and standard deviation $\sigma$.

In [ ]:
mus  = np.asarray([0,1.5])
stds = np.asarray([1,.5]) 
gstat = np.asarray([mus, stds]).T 
pi = 0.3
#g = np.asarray([st.norm(*gstat[0]),st.norm(*gstat[1])])
g = st.norm(loc=mus, scale=stds)

In [ ]:
def p_mix(x):
    pdf = g.pdf(np.atleast_1d(x).reshape(-1,1))
    return (1-pi)*pdf[:,0]+pi*pdf[:,1]

In [ ]:
xs = np.linspace(-5,5,500)
pdf = g.pdf(xs.reshape(-1,1))
plt.plot(xs, (1-pi)*pdf[:,0]+pi*pdf[:,1],label = '$\pi \cdot  p(x|\mu_1,\sigma_1) +  (1-\pi) \cdot p(x|\mu_0,\sigma_0)$')
plt.plot(xs, (1-pi)*pdf[:,0], label = '$(1-\pi) \cdot  p(x|\mu_0,\sigma_0)$');
plt.plot(xs, pi*pdf[:,1], label = '$\pi \cdot  p(x|\mu_1,\sigma_1)$');
plt.legend(loc = 2);

#### Sampling from a mixture dustribution

When sampling a mixture distribution we  do it in two steps: First we  select the component using the Bernoulli distribution (multinoulli in general when number of components is greater then two)

In [ ]:
np.random.seed(12312)
z  = st.bernoulli(p=pi)

In [ ]:
zs = z.rvs(20000)

and then we draw a sample from this component distribution

In [ ]:

data = st.norm.rvs(gstat[zs][:,0], gstat[zs][:,1])

This is called _ancestral sampling_. 

In [ ]:
plt.hist(data, bins=64, density=True);
plt.plot(xs, p_mix(xs));

### Log likelihood

We will try to fit this distribution to data using MLE. Given the data $\mathbf x$ the likelihood is

$$\newcommand{\b}[1]{\mathbf{#1}}$$
$$p(\b x|\pi, \mu_0,\sigma_0, \mu_1, \sigma_1) = \prod_{i=1}^N p(x_i|\pi, \mu_0\sigma_0, \mu_1, \sigma_1)$$

$$\log p(\b x|p, \mu_0,\sigma_0, \mu_1, \sigma_1) = \sum_{i=1}^N \log\left(
\pi \cdot p_\nc(x_i|\mu_1,\sigma_1) +  (1-\pi) \cdot p_\nc(x|\mu_0,\sigma_0)
\right)$$

This is similar to what we have encountered when fitting the quadratic discriminant model but with one crucial difference. Because we do not know which points belong to which cluster the argument to the logarithm is a sum. As we cannot apply the logarithm separately to each term we end up with much more complicated expression. What's worse the resulting function is _not concave_ as a function of parameters $\pi, \mu_0,\sigma_0, \mu_1$ and $\sigma_1$. That makes  finding the maximum estimate (MLE) much more difficult. We can of course use some universal maximum finding algorith like stochastic gradient descent but we have to ascertain that e.g. paramter $\pi$ is  contained in the interval $[0,1)$. 

Fortunatelly there exists an algorithm  that is aplicable to exactly such problems involving hidden variables, which I will describe below.

### Latent variables

Let's suppose we know to which component each data point belongs. We can encode this information into a variable $z$ that takes values $0$  and $1$. Then the _joint_ distribution of variables $x$ nad $z$ is

$$
P(x,z|\pi,\mu_0,\sigma_0, \mu_1, \sigma_1) = 
z \cdot \pi \cdot p_\nc(x|\mu_1,\sigma_1)+  (1-z)\cdot(1-\pi) \cdot p_\nc(x|\mu_0,\sigma_0)
$$

and the log likelihood is 

$$
\log P(\b x,\b z|\pi,\mu_0,\sigma_0, \mu_1, \sigma_1) = \sum_{i=1}^N z_i \log\left(
\pi \cdot p_\nc(x_i|\mu_1,\sigma_1)\right)+  (1-z_i)\log \left((1-p) \cdot p_\nc(x_i|\mu_0,\sigma_0)
\right)
$$

Now we could take the log of each term in the sum separately because only one term can be non-zero at the same time. The variables $z$ are called _latent_ or _unobserved_ variables. 

### Expectation - Maximization algorithm

The _marginal_ distribution of $z$ is 

$$P(z =1)=\int_x P(x,z=1|\pi,\mu_0,\sigma_0, \mu_1, \sigma_1) =
\pi \cdot  \int_x p_\nc(x|\mu_1,\sigma_1)=\pi
$$

The conditional probability of $z$

$$\gamma_\theta(x)\equiv P(z =1|x,\theta)$$

where $\theta$ denotes all the parameters $\pi, \mu_0,\sigma_0, \mu_1, \sigma_1$, can be easilly calculated  using Bayes theorem

$$P(z =1|x,\theta) = 
\frac{p(x|z=1,\theta)P(z=1)}
{p(x|z=0, \theta)P(z=0)+p(x|z=1,\theta)P(z=1)}
=
\frac{\pi p_\nc(x_i|\mu_1,\sigma_1)}{\pi p_\nc(x_i|\mu_1, \sigma_1)+ (1-\pi) p_\nc(x_i|\mu_0,\sigma_0)}$$

Given $\gamma_\theta(x)$ we can calculate the __expected__ log likehood 

$$Q(\theta', \theta) \equiv E_{P(\b z|\b x, \theta)}\left[\log P(\b x,\b z|\theta')\right]\equiv \sum_i \left[\gamma_\theta(x_i) \log P(\b x_i, z_i=1|\theta')+(1-\gamma_\theta(x_i))\log P(\b x_i, z_i=0|\theta')\right]$$

Please note that the expectation value was calculated  using the probability distribution for $\b z$ with parameters $\theta$. However in the  joint probability distribution being averaged $P(\b x,\b z|\theta')$ we have assumed a different set of parameters $\theta'$. The final expression is 

$$\sum_{i=1}^N \left[\gamma_\theta(x_i)\log p_\nc(x_i|\mu'_1,\sigma'_1)+  (1-\gamma_\theta(x_i))\log  p_\nc(x_i|\mu'_0,\sigma'_0)\right]
+
\sum_{i=1}^N \left[\gamma_\theta(x_i)\log \pi' +(1-\gamma_\theta(x_i))\log (1-\pi') \right]
$$

Now we can calculate the parameters $\hat\theta$ that __maximize__ this expectation value

$$\newcommand{\argmax}{\operatorname{argmax}}$$
$$\hat\theta = \argmax_{\theta'}Q(\theta', \theta)$$

Let's calculate the $\hat\pi$. Differentiating the likelihood with respect to $\pi'$ we obtain the equation for $\hat\pi$

$$\frac{1}{\hat\pi} \sum_{i=1}^N\gamma_\theta(x_i)-\frac{1}{1-\hat\pi}\sum_{i=1}^N(1-\gamma_\theta(x_i))=0$$

with solution

$$\hat\pi = \frac{1}{N}\sum_{i=1}^N\gamma_\theta(x_i)$$

$$\sum_{i=1}^N \gamma_\theta(x_i)\log p_\nc(x_i|\mu'_1,\sigma'_1) = 
-\log\sigma'_1 \sum_{i=1}^N \gamma_\theta(x_i) 
-\sum_{i=1}^N \gamma_\theta(x_i)\frac{1}{2\sigma^2}(x_i-\mu'_1)^2$$

Differentiating with respect to $\mu'_1$ we obtain the equation for $\hat\mu$

$$\frac{1}{\sigma^2}\sum_{i=1}^N \gamma_\theta(x_i)(x_i-\hat\mu_1) = 0 $$

which is equivalent to

$$\sum_{i=1}^N \gamma_\theta(x_i) x_i = \hat\mu_1\sum_{i=1}^N \gamma_\theta(x_i) $$

leading to 

$$\hat\mu_1 = \frac{\sum_{i=1}^N \gamma_\theta(x_i) x_i}{\sum_{i=1}^N \gamma_\theta(x_i)}$$

The rest of the parameters is calculated in the same way giving finally:

$$
\hat\mu_0 = \frac{\sum_{i=1}^N  (1- \gamma_\theta(x_i)) x_i}{\sum_{i=1}^N  (1-\gamma_\theta(x_i))}
\qquad
\hat\mu_1 = \frac{\sum_{i=1}^N \gamma_\theta(x_i) x_i}{\sum_{i=1}^N \gamma_\theta(x_i)}
$$

$$
\hat\sigma_0=\frac{\sum_{i=1}^N (1-\gamma_\theta(x_i)) (x_i-\hat\mu_0)^2}{\sum_{i=1}^N  (1-\gamma_\theta(x_i))}
\qquad
\hat\sigma_1=\frac{\sum_{i=1}^N \gamma_\theta(x_i) (x_i-\hat\mu_1)^2}{\sum_{i=1}^N \gamma_\theta(x_i)}
$$

Repeating this steps leads to the __Expectation-maximization__ (EM) algorithm:
  1. Start by initialising the parameters $\theta$
  2. Calculate the $\gamma_\theta(x_i)$: that is the __expectation__ step
  3. Use $\gamma$ to calculate new parameters $\hat\theta$: that's the __maximization__ step
  4. Set $\theta = \hat\theta$
  4. Repeat until convergence

[A. P. Dempster, N. M. Laird and D. B. Rubin, "Maximum Likelihood from Incomplete Data via the EM Algorithm" Journal of the Royal Statistical Society. Series B 
Vol. 39, No. 1 (1977), pp. 1-38](https://www.jstor.org/stable/2984875?seq=1#metadata_info_tab_contents)

Those steps for this simple two gaussians example are implemented below:

In [ ]:
def expectation(x,theta):
    pi, m1, s1, m2,s2 = theta
    d2 = pi*st.norm(loc=m2, scale=s2).pdf(x)
    d1 = (1-pi)*st.norm(loc=m1, scale=s1).pdf(x)
    return d2/(d1+d2)

def maximization(x,z):
    N = len(z)
    z_sum = z.sum()
    m1 = np.sum((1-z)*x)/(N-z_sum)
    m2 = np.sum(z*x)/z_sum
    
    s1 = np.sqrt(np.sum((1-z)*(x-m1)*(x-m1))/(N-z_sum))
    s2 = np.sqrt(np.sum(z*(x-m2)*(x-m2))/z_sum)
    
    pi = z_sum/N
    
    return np.asarray([pi,m1,s1,m2,s2])
 
def next_theta(x, theta):
    z = expectation(x,theta)
    return maximization(x,z)

In [ ]:
# Starting parameters
# For mu I choose two data points at random
start = np.random.choice(data,2, replace=False)
# For sigma I use the std of the whole dataset
theta = np.asarray([0.5, start[0], data.std(), start[1], data.std()])

In [ ]:
#Collect the results of 300 iterations
ts = [theta]
for i in range(300):
    theta = next_theta(data,theta)
    ts.append(theta)
thetas = np.stack(ts, axis=0)    

Below are plots showing the convergence of all parameters.

In [ ]:
plt.scatter(range(len(thetas)), thetas[:,0], s=10, alpha=0.5, label='$\pi$');
plt.axhline(0.3);
plt.axhline(0.7);
plt.legend();

In [ ]:
plt.scatter(range(len(thetas)), thetas[:,1], s=10, alpha=0.5, label = "$\mu_1$", color = 'blue');
plt.axhline(mus[0], color = 'blue')
plt.scatter(range(len(thetas)), thetas[:,3], s=10, alpha=0.5, label = "$\mu_2$", color = 'orange');
plt.axhline(mus[1], color='orange');
plt.legend();

In [ ]:
plt.scatter(range(len(thetas)), thetas[:,2], s=10, alpha=0.5, label = "$\sigma_1$", color = 'blue');
plt.axhline(stds[0], color = 'blue')
plt.scatter(range(len(thetas)), thetas[:,4], s=10, alpha=0.5, label = "$\sigma_2$", color = 'orange');
plt.axhline(stds[1], color='orange');
plt.legend();

The algorithm is garantied (within numericall accuracy) never to decrease the likelihood, so let's check this

In [ ]:
def loglikehood(x,theta):
    pi, m1, s1, m2,s2 = theta
    d2 = pi * st.norm(loc=m2, scale=s2).pdf(x)
    d1 = (1-pi) * st.norm(loc=m1, scale=s1).pdf(x)
    return np.log(d1 + d2).sum()

In [ ]:
ll = [loglikehood(data,t) for t in thetas]

In [ ]:
plt.scatter(range(len(ll)), ll, s=5, alpha = 0.5);

As you can see we gain most from the first step. 

Below is a  simple animation of the whole process

In [ ]:
%%capture
from matplotlib.animation import FuncAnimation
xs = np.linspace(-5,5,500)
fig, ax = plt.subplots()
ax.set_ylim(0,0.4)
l1, = plt.plot([-5,5],[0,0])
l2, = plt.plot([-5,5],[0,0])
l3, = plt.plot([-5,5],[0,0])
plt.hist(data, bins=64,  density=True, histtype='step')


def animate(theta):
    pi, m1, s1, m2,s2 = theta
    d2 = pi*st.norm(loc=m2, scale=s2).pdf(xs)
    d1 = (1-pi)*st.norm(loc=m1, scale=s1).pdf(xs)
    l1.set_data(xs,d1)
    l2.set_data(xs,d2)
    l3.set_data(xs,d1+d2)
    
    
anim = FuncAnimation(fig, animate, thetas, repeat=False, interval=40)    

In [ ]:
## Can take ~15 seconds to prepare
anim

### Notes on the convergence

#### Unidentifiability

Looking at the model mixture distribution function it's easy to notice that it is independent  with respect to the permutations of the parameters i.e. if we exchange $\pi\leftrightarrow (1-\pi)$, $\mu_0\leftrightarrow \mu_1$ and $\sigma_0\leftrightarrow \sigma_1$  the resulting pdf will be the same.  That's called _unidentifiability_. You can observe this on the convergence plots above. Different components are marked in different colors and as you can see the colors may not correspond to  the colors of the true values.

#### Maximum

The EM algorithm is guaranteed not to decrease the likelihood at each step, so it will usually converge to some local maxima. One can start the algorithm with different initial parameters and choose the result with highest likelihood. 

Actually the fact that the algorithm will rather find local then global maximum is a good thing. For mixture of gaussian the global maximum is degenerate (!) and can be obtained as follows: pick one point $x_0$ in the data and consider $\mu_0=x_0$ and $\sigma_0\rightarrow 0$. In this limit the contribution from this  single point to the likelihood will be infinite (why?) , while the other component will provide finite values for the rest of the data points. 

This behaviour is illustrated below. This have been tuned to this particular dataset. I have fixed the seed when generating data, but I am not sure if it is portable across all the numpy versions, so you may need to experiment. 

Unfortunatelly the functions  defined above will fail because of the numerical instabilities. That is a common problem when dealing with probabilities that can have expotentially small values. The problem arises when we have small values in both numerator and denominator. The solution is to use logarithms and pull out the biggest term in the sum both in numerator and denominator. This is imlemented in the function below

In [ ]:
ds = data[:50]

In [ ]:
def expectation_stable(x,theta):
    pi, m1, s1, m2,s2 = theta
    d = st.norm([m1,m2],[s1,s2]).logpdf(x.reshape(-1,1))+np.log(np.asarray([[1-pi, pi]]))
    d_max = np.max(d,1)
    
    return np.exp(d[:,1]-d_max)/np.sum(np.exp(d-d_max.reshape(-1,1)),1)    

In [ ]:
#Some helper functions that will be used to break the iterations  when we reach a singularity
def is_nan(t):
    return np.any(np.isnan(t)) 

def valid_thetas(th):
    not_nan = ~np.any(np.isnan(thetas),1) 
    v_th = th[not_nan]
    s_th = (v_th[:,2]>1.0e-10) & (v_th[:,4]>1.0e-10)
    return v_th[s_th]

In [ ]:
theta = [0.5, ds[3],  0.4, ds[9], 0.121]

In [ ]:
res = [theta]
for i in range(100):
    z = expectation_stable(ds,theta)
    theta = maximization(ds,z)
    if  is_nan(theta):
        break
    res.append(theta)
thetas = np.stack(res,0)    

In [ ]:
plt.scatter(np.arange(len(thetas)), thetas[:,2],s=5,label='$\\sigma_0$')
plt.scatter(np.arange(len(thetas)), thetas[:,4],s=5,label='$\\sigma_1$');
plt.legend();

If everything went allright with the example you should see on of the $\sigma$'s go to zero on this plot.  This corresponds to one of the distributions collapsing onto one point as illustrated in the plot below.

In [ ]:
def plot(theta,ax):
    xs = np.linspace(-4,4,5000)
    ax.set_ylim(0,2.75)
    pi, m1, s1, m2,s2 = theta
    d2 = pi*st.norm(loc=m2, scale=s2).pdf(xs)
    d1 = (1-pi)*st.norm(loc=m1, scale=s1).pdf(xs)
    ax.plot(xs,d1,c='red')
    ax.plot(xs,d2,c='blue')
    #plt.plot(xs,d1+d2,c='green')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(18,6))
plot(thetas[5],ax[0])
plot(thetas[6],ax[1])
plot(thetas[7],ax[2])

This behaviour will be much more pronounced in higher dimensions. That is because with same number of points, the points in higher dimensions will be on average more distanced from each other.  Thus it is easier to "isolate" one point which leads to the described singularity. 

###  MAP  estimation

This problem can be avoided by  using the full Bayesian approach. To this end we add prior for $\pi$ and  for parameters $\mu_k$ and $\sigma_k$. For the EM algorithm this amounts to changing the expected log likelihood to

$$\begin{multline}
\sum_{i=1}^N \left[\gamma_\theta(x_i)\log p_\nc(x_i|\mu'_1,\sigma'_1)+  (1-\gamma_\theta(x_i))\log  p_\nc(x_i|\mu'_0,\sigma'_0)\right] +\log P(\pi) +\log P(1-\pi)
+\\
\sum_{i=1}^N \left[\gamma_\theta(x_i)\log \pi' +(1-\gamma_\theta(x_i))\log (1-\pi') \right]
+\log P(\mu_0,\sigma_0) +\log P(\mu_1, \sigma_1)
\end{multline}
$$

For $\pi$ we chose  [Beta](https://en.wikipedia.org/wiki/Beta_distribution) distribution prior and  [Normal Inverse Gamma](https://en.wikipedia.org/wiki/Normal-inverse-gamma_distribution) prior for parameters $\mu_k$ and $\sigma_k$.  Fitting with those priors is described in detail in the normal distribution notebook. The result is 

$$\hat\pi = \frac{\sum_{i=1}^N\gamma_\theta(x_i)+\alpha-1}{N+\alpha+\beta-2}$$

$$
\hat\mu_0 = \frac{\sum_{i=1}^N  (1- \gamma_\theta(x_i)) x_i +\lambda_0 m_0}{\sum_{i=1}^N  (1-\gamma_\theta(x_i))+\lambda_0}
\qquad
\hat\mu_1 = \frac{\sum_{i=1}^N \gamma_\theta(x_i) x_i+\lambda_1 m_1}{\sum_{i=1}^N \gamma_\theta(x_i)+\lambda_1}
$$

$$
\hat\sigma_0=\frac{\sum_{i=1}^N (1-\gamma_\theta(x_i)) (x_i-\hat\mu_0)^2+2\beta_0 
+\lambda_0(\hat\mu_0-m)^2}
{\sum_{i=1}^N  (1-\gamma_\theta(x_i))+2\alpha_0 + 3}
$$

$$
\hat\sigma_1=\frac{\sum_{i=1}^N \gamma_\theta(x_i) (x_i-\hat\mu_1)^2+
2\beta_1 +\lambda_1(\hat\mu_1-m_1)^2}
{\sum_{i=1}^N \gamma_\theta(x_i)+2\alpha_1 + 3}
$$

That introduces 10 (!) more hyperparameters: $\alpha$  and $\beta$ for Beta distribution and $m_k,\lambda_k, \alpha_k,\beta_k$ for two Normal Inverse  Gamma priors on $\mu_k$ and $\sigma_k$ ($k=0,1$). 
We can simplify things assuming a symmetric prior on $\pi$ with $\alpha=\beta$ leading to

$$\hat\pi = \frac{\sum_{i=1}^N\gamma_\theta(x_i)+\alpha-1}{N+2\alpha-2}$$

$\alpha$ smaller then one will tend to concentrate the probability mass on one of the components and higher values of  $\alpha$ will favor more uniform  distribution. 

Settinf $\lambda_k = 0$ effectively removes the prior on the $\mu_k$ 

$$
\hat\mu_0 = \frac{\sum_{i=1}^N  (1- \gamma_\theta(x_i)) x_i}{\sum_{i=1}^N  (1-\gamma_\theta(x_i))}
\qquad
\hat\mu_1 = \frac{\sum_{i=1}^N \gamma_\theta(x_i) x_i}{\sum_{i=1}^N \gamma_\theta(x_i)}
$$

and finally we will set $\alpha_k$ to $3/2$ resulting in: 

$$
\hat\sigma_0=\frac{\sum_{i=1}^N (1-\gamma_\theta(x_i)) (x_i-\hat\mu_0)^2+2\beta_0}
{\sum_{i=1}^N  (1-\gamma_\theta(x_i))+2\alpha_0 + 3}
\quad
\hat\sigma_1=\frac{\sum_{i=1}^N \gamma_\theta(x_i) (x_i-\hat\mu_1)^2+
2\beta_1}
{\sum_{i=1}^N \gamma_\theta(x_i)+2\alpha_1 + 3}
$$

That leaves us  with only three additional parameters. Those formulas are implemented in funnctions below

In [ ]:
def MAP(x,z, a,b0, b1):
    N = len(z)
    z_sum = z.sum()
    m1 = np.sum((1-z)*x)/(N-z_sum)
    m2 = np.sum(z*x)/z_sum
    
    s1 = np.sqrt((np.sum((1-z)*(x-m1)*(x-m1))+2*b0)/(N-z_sum+6))
    s2 = np.sqrt((np.sum(z*(x-m2)*(x-m2))+2*b1)/(z_sum+6))
    
    pi = (z_sum+a-1)/(N+2*a-2)
    
    return np.asarray([pi,m1,s1,m2,s2])
 
def next_theta_MAP(x, theta,a,b0,b1):
    z = expectation_stable(x,theta)
    return MAP(x,z,a,b0,b1)

I leave it to you to check  they behaviour.

## Half circles revisited

Armed with the EM algorithm we now come back to the half circles example. However we will not write our own EM algorithm for multivariate normal distribution, but we will use an implementation from scikit-learn library provided by the `GaussianMixture` class

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
hc0_cmp = GaussianMixture(n_components=2, max_iter=100, tol=0.0001) 
hc1_cmp = GaussianMixture(n_components=2, max_iter=100, tol=0.0001) 

In [ ]:
hc0 = hc_train[hc_lbl_train==0]
hc1 = hc_train[hc_lbl_train==1]

In [ ]:
hc0_cmp.fit(hc0)
hc1_cmp.fit(hc1)

The fitted parameters are accesible as attributes of the `GaussianMixture` object

In [ ]:
print(hc0_cmp.weights_)  #pi
print(hc0_cmp.means_)    #mu
print(hc0_cmp.covariances_) #Sigma (covariance) matrices

Now we will use those fitted mixtures to construct a classifier

In [ ]:
def make_pdf(cmp):
    """
    Takes a GaussianMixture object and returns corresponding
    probability distribution function
    """
    n_cmp = cmp.n_components
    dists = [st.multivariate_normal(cmp.means_[i], cmp.covariances_[i]) for i in range(n_cmp)]
    def pdf(x):
        p = 0.0
        for i in range(n_cmp):
            p+= cmp.weights_[i]*dists[i].pdf(x)
        return p
    
    return pdf
    
    
def make_predict_proba(cmp0, cmp1, pi0=0.5, pi1=.5):
    """
    Takes two GaussianMixture object and corresponding priors and returns 
    pdf for conditional probability P(c=1|x)
    """
    pdf0 = make_pdf(cmp0)
    pdf1 = make_pdf(cmp1)
    def p(x):
        p0=pi0*pdf0(x)
        p1=pi1*pdf1(x)
        return p1/(p1+p0)    
        
    return p
        

In [ ]:
mgd_predict_proba = make_predict_proba(hc0_cmp, hc1_cmp, 0.5, 0.5)

In [ ]:
mgd_proba = mgd_predict_proba(hc_test)

In [ ]:
confusion_matrix(hc_lbl_test, mgd_proba>0.5, normalize='true')

In [ ]:
from mchlearn.plotting import roc_plot, add_roc_curve
fig, ax = roc_plot()
add_roc_curve(hc_lbl_test, qda_proba, 'qda', ax);
add_roc_curve(hc_lbl_test, mgd_proba, 'mga', ax);
ax.legend(title='AUC');

We see a dramatic improvement in the quality of the classifier.  Which is explainable by looking at the fitted components and the new decision boundary

In [ ]:
fig, ax = plt.subplots()
blue_red = np.asarray([[0., 0.,1.],[1.,0.,0.]]) 
decision_plot(half_circles[:,:2], half_circles[:,2].astype('int32'), blue_red, np.linspace(-2.5,2.5,200), np.linspace(-1.5,1.5,200),
              mgd_predict_proba, lambda x: 0 if x<0.5 else 1  , ax = ax)
confidence_ellipse(hc0_cmp.means_[0], hc0_cmp.covariances_[0], ax = ax, edgecolor='blue')
confidence_ellipse(hc0_cmp.means_[1], hc0_cmp.covariances_[1], ax = ax, edgecolor='blue')
confidence_ellipse(hc1_cmp.means_[0], hc1_cmp.covariances_[0], ax = ax, edgecolor='red')
confidence_ellipse(hc1_cmp.means_[1], hc1_cmp.covariances_[1], ax = ax, edgecolor='red');